In [32]:
import traffic
import pandas as pd 
import os
from traffic.core import Traffic
from traffic.data import opensky 
from biblio_herramienta.herramienta import *
from biblio_herramienta.tratardatos import *
from biblio_herramienta.conflictos import *

In [33]:
carpetaDatos = r'datos_sectores/' # carpeta que contiene los datos
#nombreDatos = r"bilbao_f_cluster.csv"
nombreDatos = r"datos_javi.csv"
carpetaImagenes = r"generacion_conflictos/"

In [34]:
vuelos = cargardatos(carpetaDatos,nombreDatos)
vuelos.data.info()

/Users/jaimebowen/opt/miniconda3/envs/tfgpaquetes/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196706 entries, 0 to 196705
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   Unnamed: 0     196706 non-null  int64              
 1   alert          196706 non-null  bool               
 2   altitude       137034 non-null  float64            
 3   callsign       190053 non-null  object             
 4   geoaltitude    120760 non-null  float64            
 5   groundspeed    166308 non-null  float64            
 6   hour           196706 non-null  datetime64[ns, UTC]
 7   icao24         196706 non-null  object             
 8   last_position  196706 non-null  datetime64[ns, UTC]
 9   latitude       196706 non-null  float64            
 10  longitude      196706 non-null  float64            
 11  onground       196706 non-null  bool               
 12  spi            196706 non-null  bool               
 13  squawk         169615 non-nul

In [35]:
def filtrardatos2(datos):
    if not isinstance(datos,Traffic):
        datos = Traffic(datos)
    # condiciones del sector
    aviones_entierra =  datos.data[datos.data["onground"] == True].index 
    datos.data.drop(aviones_entierra,inplace = True)
    

    # filtrado de datos no validos
    datos = datos.clean_invalid()
    datos = datos.drop_duplicates()

    # filtrado de datos incompletos
    Var_huecos = datos.data.columns[datos.data.isnull().any()]
    # identificacion de filas con NaN
    nan_rows = datos.data[datos.data.isnull().any(1)]

    # identificacion de aeronaves con NaN
    ave_huecos = nan_rows.callsign.value_counts()
    list_ave = []
    # identificacion de aeronaves con >10 fallos de recepción ADS-B
    v_filtrado = datos
    for count, values in enumerate(ave_huecos):
        if values > 10:
            list_ave.append(ave_huecos.index[count])
            # se identifican las filas de las aeronaves y las eliminamos
            rows = datos.data.loc[datos.data.callsign == ave_huecos.index[count]].index
            v_filtrado = v_filtrado.drop(rows)

    # se eliminan todas las filas que tienen NaN
    v_filtrado = v_filtrado.data.dropna() # OJO devuelve un df no un Traffic
    v_filtrado = traffic.core.Traffic(v_filtrado)
    Var_huecos = []
    # identificación de aeronaves con huecos
    aves_huecos = []
    # se cambian los nans por la media enter los dos valores
    from sklearn.impute import SimpleImputer
    my_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

    #Aplicamos el método a cada aeronave:
    for count, name in enumerate(aves_huecos):
        ave_impute = pd.DataFrame(my_imputer.fit_transform(vuelos_prueba))
    return v_filtrado

In [36]:
vuelos = filtrardatos2(vuelos)
vuelos = vuelos.assign_id().unwrap().eval(max_workers=4)
guardarcsv(vuelos,"datos_javi_filtrado")


In [37]:
vuelos.data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66743 entries, 0 to 276
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   Unnamed: 0       66743 non-null  int64              
 1   alert            66743 non-null  bool               
 2   altitude         66743 non-null  float64            
 3   callsign         66743 non-null  object             
 4   geoaltitude      66743 non-null  float64            
 5   groundspeed      66743 non-null  float64            
 6   hour             66743 non-null  datetime64[ns, UTC]
 7   icao24           66743 non-null  object             
 8   last_position    66743 non-null  datetime64[ns, UTC]
 9   latitude         66743 non-null  float64            
 10  longitude        66743 non-null  float64            
 11  onground         66743 non-null  bool               
 12  spi              66743 non-null  bool               
 13  squawk           6